In [1]:
from IPython.display import HTML
HTML('<iframe src="http://pydata.org/nyc2014/schedule/" width=100% height=350></iframe>')

In [6]:
from bs4 import BeautifulSoup
import urllib2

# List of conference schedule pages
urls = {'nyc2014' : "http://pydata.org/nyc2014/abstracts/",
        }

links = {}

# Collecting the links to the talk abstracts
for u in urls:
    raw = urllib2.urlopen(urls[u]).read()
    soup = BeautifulSoup(raw)
    yearlinks = set([l.get("href") for l in soup.find_all("a")])
    yearlinks = [l for l in yearlinks if not (l is None)]
    yearlinks = [l for l in yearlinks if '/detail' in l]
    yearlinks = [l.replace("http://strataconf.com", "") for l in yearlinks]
    links[u] = yearlinks

In [8]:
url = 'http://pydata.org/nyc2014/abstracts/' 

In [10]:
raw = urllib2.urlopen(url).read()

In [12]:
soup = BeautifulSoup(raw)


In [29]:
abstracts = [abstract.get_text() for abstract in soup.find_all(class_="accordion-inner")]

In [31]:
abstracts

[u'\n                    This talk covers rapid prototyping of a high performance scalable text processing pipeline development in Python. We demonstrate how Python modules, in particular from the Rosetta library, can be used to analyze, clean, extract features, and finally perform machine learning tasks such as classification or topic modeling on millions of documents. Our style is to build small and simple modules (each with command line interfaces) that use very little memory and are parallelized with the multiprocessing library.\n                ',
 u'\n                    Scikit-Learn is one of the most popular machine learning library written in Python, it has quite active community and extensive coverage for a number of machine learning algorithms. It has feature extraction, feature and model selection algorithms, and validation methods as well to build a modern machine learning pipeline. \r\n\r\nThis tutorial introduces common recipes to build a modern machine learning pipeline

In [32]:
import nltk
bigram_measures = nltk.collocations.BigramAssocMeasures()
stop = nltk.corpus.stopwords.words('english')

text = {}
words = {}

In [34]:
raw = " ".join(abstracts)
tokens = nltk.WordPunctTokenizer().tokenize(raw)
text = nltk.Text(tokens)

In [36]:
text.collocations()

machine learning; open source; data science; Cloud Foundry; command
line; Coming soon; time series; financial statements; New York; Time
permitting; confidence pool; crafted artisanal; dimensionality
reduction; gene expression; keep track; rapid prototyping; big data;
http ://; style buildpack; web application


In [37]:
from nltk.collocations import *

bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

In [47]:
words = [w.lower() for w in text]
words = [w for w in words if w not in stop]
words = filter(lambda word: word not in u'%,-:()$\/;?.’–“”!\'[', words)
words = [w for w in words if w not in ["ll", "II", "ll", "http", "://", "e", "g", "2", "0", "1"]]
    

In [44]:
words

[u'talk',
 u'covers',
 u'rapid',
 u'prototyping',
 u'high',
 u'performance',
 u'scalable',
 u'text',
 u'processing',
 u'pipeline',
 u'development',
 u'python',
 u'demonstrate',
 u'python',
 u'modules',
 u'particular',
 u'rosetta',
 u'library',
 u'used',
 u'analyze',
 u'clean',
 u'extract',
 u'features',
 u'finally',
 u'perform',
 u'machine',
 u'learning',
 u'tasks',
 u'classification',
 u'topic',
 u'modeling',
 u'millions',
 u'documents',
 u'style',
 u'build',
 u'small',
 u'simple',
 u'modules',
 u'command',
 u'line',
 u'interfaces',
 u'use',
 u'little',
 u'memory',
 u'parallelized',
 u'multiprocessing',
 u'library',
 u'scikit',
 u'learn',
 u'one',
 u'popular',
 u'machine',
 u'learning',
 u'library',
 u'written',
 u'python',
 u'quite',
 u'active',
 u'community',
 u'extensive',
 u'coverage',
 u'number',
 u'machine',
 u'learning',
 u'algorithms',
 u'feature',
 u'extraction',
 u'feature',
 u'model',
 u'selection',
 u'algorithms',
 u'validation',
 u'methods',
 u'well',
 u'build',
 u'modern

In [48]:
finder = BigramCollocationFinder.from_words(words)
scored = finder.score_ngrams(bigram_measures.raw_freq)

In [49]:
scored

[((u'machine', u'learning'), 0.004191616766467066),
 ((u'data', u'science'), 0.0032934131736526945),
 ((u'scikit', u'learn'), 0.0032934131736526945),
 ((u'open', u'source'), 0.002694610778443114),
 ((u'big', u'data'), 0.0014970059880239522),
 ((u'coming', u'soon'), 0.0011976047904191617),
 ((u'data', u'processing'), 0.0011976047904191617),
 ((u'data', u'sets'), 0.0011976047904191617),
 ((u'python', u'ecosystem'), 0.0011976047904191617),
 ((u'cloud', u'foundry'), 0.0008982035928143712),
 ((u'command', u'line'), 0.0008982035928143712),
 ((u'data', u'analysis'), 0.0008982035928143712),
 ((u'deep', u'learning'), 0.0008982035928143712),
 ((u'financial', u'statements'), 0.0008982035928143712),
 ((u'languages', u'like'), 0.0008982035928143712),
 ((u'learning', u'techniques'), 0.0008982035928143712),
 ((u'light', u'weight'), 0.0008982035928143712),
 ((u'one', u'might'), 0.0008982035928143712),
 ((u'selective', u'search'), 0.0008982035928143712),
 ((u'show', u'one'), 0.0008982035928143712),
 ((